In [ ]:
!pip install profanity-check
!pip install nltk
!pip install numpy
!pip install transformers
!pip install conceptnet
!pip install better_profanity
!pip install nrclex


In [20]:
from conceptnet5.uri import join_uri, split_uri
import requests

# Example query
concept_uri = '/c/en/hi'
api_url = f'http://api.conceptnet.io{concept_uri}'

# Send HTTP GET request to ConceptNet API
response = requests.get(api_url)

# Process the response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f'Error: Failed to fetch data. Status code: {response.status_code}')


{'@context': ['http://api.conceptnet.io/ld/conceptnet5.7/context.ld.json'], '@id': '/c/en/hi', 'edges': [{'@id': '/a/[/r/Synonym/,/c/ja/ホームインプルーブメント/n/,/c/en/hi/]', '@type': 'Edge', 'dataset': '/d/jmdict', 'end': {'@id': '/c/en/hi', '@type': 'Node', 'label': 'hi', 'language': 'en', 'term': '/c/en/hi'}, 'license': 'cc:by-sa/4.0', 'rel': {'@id': '/r/Synonym', '@type': 'Relation', 'label': 'Synonym'}, 'sources': [{'@id': '/s/resource/jmdict/1.07', '@type': 'Source', 'contributor': '/s/resource/jmdict/1.07'}], 'start': {'@id': '/c/ja/ホームインプルーブメント/n', '@type': 'Node', 'label': 'ホームインプルーブメント', 'language': 'ja', 'sense_label': 'n', 'term': '/c/ja/ホームインプルーブメント'}, 'surfaceText': None, 'weight': 2.0}, {'@id': '/a/[/r/Synonym/,/c/en/hi/,/c/sh/bok/]', '@type': 'Edge', 'dataset': '/d/wiktionary/en', 'end': {'@id': '/c/sh/bok', '@type': 'Node', 'label': 'bok', 'language': 'sh', 'term': '/c/sh/bok'}, 'license': 'cc:by-sa/4.0', 'rel': {'@id': '/r/Synonym', '@type': 'Relation', 'label': 'Synonym'}, 'so

In [38]:
related_words = set()
for edge in data['edges']:
            relation=edge['rel']['label']
            if relation in ['RelatedTo', 'IsA', 'HasA']:
              related_words.add(edge['end']['label'])



print(list(related_words))


['hi']


In [50]:
from conceptnet5.uri import join_uri, split_uri
import requests

def get_related_words(word):
  concept_uri='/c/en/'+word
  api_url = f'http://api.conceptnet.io{concept_uri}'
  relations = ['relatedto','formof','isa','partof','usedfor','capableof','causes','hassubevent','hasfirstsubevent','hasprerequisite','hasproperty','synonym','antonym','distinctfrom','derivedfrom','definedas','hascontext','similarto','etymologicallyrelatedto','receivesaction']


  # Send HTTP GET request to ConceptNet API
  response = requests.get(api_url)

  # Process the response
  if response.status_code == 200:
    data = response.json()
  else:
    print(f'Error: Failed to fetch data. Status code: {response.status_code}')
    return 0

  related_words = set()
  for edge in data['edges']:
            relation=edge['rel']['label']
            if relation.lower() in relations:

              related_words.add(edge['end']['label'])

  print(list(related_words))

get_related_words('life')

['finite', 'life', 'living', 'good']


In [59]:
import random
from conceptnet5.uri import split_uri
from transformers import pipeline
from better_profanity import profanity
from nrclex import NRCLex
import nltk
nltk.download('punkt')


# def get_related_words(word):
#     related_words = set()
#     for source in ['/c/en/', '/c/en/']:
#         query = lookup('/c/en/' + word)
#         for edge in query.edges:
#             _, relation, label = split_uri(edge['@id'])
#             if relation in ['RelatedTo', 'IsA', 'HasA']:
#                 related_words.add(label.replace('_', ' '))
#     return list(related_words)



from conceptnet5.uri import join_uri, split_uri
import requests

def get_related_words(word):
  concept_uri='/c/en/'+word
  api_url = f'http://api.conceptnet.io{concept_uri}'
  relations = ['relatedto','formof','isa','partof','usedfor','capableof','causes','hassubevent','hasfirstsubevent','hasprerequisite','hasproperty','synonym','antonym','distinctfrom','derivedfrom','definedas','hascontext','similarto','etymologicallyrelatedto','receivesaction']


  # Send HTTP GET request to ConceptNet API
  response = requests.get(api_url)

  # Process the response
  if response.status_code == 200:
    data = response.json()
  else:
    print(f'Error: Failed to fetch data. Status code: {response.status_code}')
    return 0

  related_words = set()
  for edge in data['edges']:
            relation=edge['rel']['label']
            if relation.lower() in relations:

              related_words.add(edge['end']['label'])

  print(list(related_words))
  return list(related_words)





def get_emotion_words(word, emotion):
    # list of words to return
    emotion_words = []
    no_emotions = []

    # check for profanity in word
    if profanity.contains_profanity(word):
        print('Sorry, there is no information for this word')
        return []

    # get associated words
    usecases = get_related_words(word)

    # check emotion of each word
    for word in usecases:
        emo_dict = NRCLex(word).affect_frequencies

        # add word to emotion_words list if it has the specified emotion
        if emotion in emo_dict.keys() and emo_dict[emotion] > 0:
            emotion_words.append(word)
        else:
            no_emotions.append(word)

    if len(emotion_words) == 0:
        return no_emotions
    else:
        return emotion_words

def generate_story_with_transformer(prompt, character, emotion):
    # Load the pre-trained GPT model
    generator = pipeline("text-generation", model="gpt2")

    # Generate text based on the prompt
    story = generator(prompt, max_length=200, num_return_sequences=1)[0]['generated_text']
    return story

def generate_story(word, emotion, character):
    # Define a prompt using the keyword and emotion
    prompt = f"Generate a story advertising {word} with an emotion of {emotion} with characteristics of {character}."

    # Generate story with the transformer
    story = generate_story_with_transformer(prompt, character, emotion)
    return story

def get_story(word, emotion):
    stories = []
    words = get_emotion_words(word, emotion)
    if len(words) > 10:
        ten_words = random.sample(words, 10)
        for i in ten_words:
            stories.append(generate_story(word, emotion, i))
    else:
        for i in words:
            stories.append(generate_story(word, emotion, i))
    return stories

if __name__ == '__main__':
    word = input("Enter a word: ")
    emotion = input("Enter an emotion: ")

    output_story = get_story(word, emotion)
    output_words = get_emotion_words(word, emotion)

    print("Output Story:")
    for story in output_story:
        print(story)

    print("\nOutput Words:")
    print(output_words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Enter a word: life
Enter an emotion: happy
['finite', 'life', 'living', 'good']


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=

['finite', 'life', 'living', 'good']
Output Story:
Generate a story advertising life with an emotion of happy with characteristics of finite.

Fiction and Video Games

The Star Citizen Story in Fallout 3

The "Gears, I'm Proud Yours", originally published by Fallout Publishing and updated by LucasArts in 2009, is a humorous parody of the "Hobby" title. The game uses a different set of elements as well, such as a "Goggles" that can be purchased for a set percentage of his (or her) health points. Their purpose is to give players something to think at, but their main purpose is to prevent player progress from starting again after a game ends.

In "The Battle-Forsaken" in Fallout 3, the Fallout universe has a "Heroic Battle-Forsaken" system, where both the player and the "victim" of a battle are rewarded with some real life equipment.[1]

In Fallout: New Vegas, this system was extended
Generate a story advertising life with an emotion of happy with characteristics of life.

Choose your sty